# Draft

In the following notebook I finally get around to trying out
the [Frames library](https://github.com/acowley/Frames), which is
an attempt to provide data frame-like capabilities (from 
[R](http://www.r-tutor.com/r-introduction/data-frame)
and 
[pandas/Python](http://pandas.pydata.org/))
in Haskell. This library is quite new, and at the time
of writing, is not available on Hackage; a direct
installation via GitHub is required:

```
% git clone https://github.com/acowley/Frames.git
% cabal sandbox add-source Frames
% cabal install Frames foldl lens-family --dry-run
% cabal install Frames foldl lens-family
```

The following is stronly based on the
[Frames tutorial](http://acowley.github.io/Frames/), but using
the ARF from 
[the previous notebook](http://htmlpreview.github.io/?https://raw.githubusercontent.com/DougBurke/astro-haskell/master/html/a%20FITSfull%20of%20ARF.html)
as the data source. As I haven't *quite* got around to writing
a full FITS parser for Haskell just yet, I am going to "cheat"
and convert the ARF to a CSV file using tools from 
[CIAO](http://cxc.harvard.edu/ciao/):

 - copy the FITS file (`src.arf`) to an ASCII "almost-CSV" file (`arf.csv`)
 
```
% dmcopy src.arf "arf.csv[opt kernel=text,sep=',']"
% head -5 arf.csv 
#TEXT/SIMPLE
#,ENERG_LO,ENERG_HI,SPECRESP
0.3000000, 0.3100000, 4.874343
0.3100000, 0.3200000, 14.82926
0.3200000, 0.3300000, 21.30229
```

 - manually edit the file to remove the first line and fix up the column names

```
% head -5 arf.csv 
ENERG_LO,ENERG_HI,SPECRESP
0.3000000, 0.3100000, 4.874343
0.3100000, 0.3200000, 14.82926
0.3200000, 0.3300000, 21.30229
0.3300000, 0.3400000, 28.51495
```

For the purposes of this notebook the file is accessed as `../data/arf.csv`.

Using the Frames library requires a selection of
[GHC extensions](https://downloads.haskell.org/~ghc/7.8-latest/docs/html/users_guide/ghc-language-features.html).
When used in source code the form

```
{-# LANGUAGE extension1, ..., extensionN #-}
```

is used, but for the IHaskell notebook it's a bunch of `:set -Xextension1` calls. It's not obvious if I need
all these in this notebook.

In [1]:
-- taken from the tutorial: http://acowley.github.io/Frames/

{-# LANGUAGE ConstraintKinds, DataKinds, FlexibleContexts, GADTs,
             OverloadedStrings, PatternSynonyms, QuasiQuotes,
             ScopedTypeVariables, TemplateHaskell, TypeOperators,
             ViewPatterns #-}

:set -XConstraintKinds
:set -XDataKinds
-- :set -XFlexibleContexts
:set -XGADTs
-- :set -XOverloadedStrings
-- :set -XPatternSynonyms
-- :set -XQuasiQuotes
-- :set -XScopedTypeVariables
-- :set -XTemplateHaskell
-- :set -XTypeOperators
-- :set -XViewPatterns

Here are some of the modules that I will use later on:

In [2]:
import qualified Control.Foldl as L
import qualified Data.Foldable as F

import Control.Applicative ((<$>), (<*>), pure)

import Data.Proxy (Proxy(..))

import Lens.Family (view)

import Frames
import Frames.CSV (readTableOpt)

The "trick" of the Frames package - i.e. the way that it provides a typed data frame - is that
data loading is separated into two parts:

  - determination of the column names and column types (e.g. the "schema" for the data)
  - reading in the data using this "schema" (this can be done in a "streaming" manner,
    i.e. chunks of rows at a time to support handling very-large files, or all the
    data can be read in one go)

So, the "evaluation" of the CSV file is handled by the 
`tableTypes` function, which is given the name of the type which will
be used to represent a row, and the path to the CSV file (there's
a variant where more control is provided to the user, but I'm using
the "easy" version here).

In [3]:
tableTypes "ARF" "../data/arf.csv"

Once this has succeeded, the compiler will create an `ARF` type that
represents the data in the file. This is handled by
[Template Haskell](https://wiki.haskell.org/Template_Haskell),
which is a GHC extension that provides compile-time metaprogramming
to Haskell. Since I am using `IHaskell`, the fact that this is done
compile-time versus evaluation-time is somewhat hidden from me
(until, if you are anything like me, you accidentally give `tableTypes` an invalid file path
and then wonder why the Haskell kernel dies on you!).

The first argument has to be capitalized, since it represents the
name of a Haskell type (which are syntactically restricted to
starting with a capital letter). Let's look at what the compiler
has worked out for the `ARF` type:

In [4]:
:opt no-pager

In [5]:
:info ARF

type ARF = Record '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double] -- Defined at :1:1

So, a row is being modelled by a `Record` with what appears to be a list$^\dagger$ of three
`name :-> type` values, which is how "named" fields are represented in `Frames`.
This is a different approach to the 
[standard Haskell record type](http://learnyouahaskell.com/making-our-own-types-and-typeclasses#record-syntax)
which I used in the previous notebook
to represent XXX-sometihng-or-other-very-interesting-XXX.

Some form of [Heterogenous lists](https://wiki.haskell.org/Heterogenous_collections)

XXX

$^\dagger$ This is not a "normal" list since it starts with "`'[`" rather than just "`[`". The
single quote character makes all the difference, and indicates that this is a
type-level list. If you look closely, you may just spot a few more single
quote characters popping up in this notebook.

The names are the column names, and their types are all
`Double`, which suggests that it's recognized that eash row has three elements:
`ENERG_LO`, `ENERG_HI`, and `SPECRESP`, each of type `Double`.

XXX comment on "list-like" and Record

XXX got to here

The `ARF` type is not the only thing that the call to `tableTypes` 
has created. One of the other items is the set of options needed
to parse the CSV file. In this case, the generated name is 
`aRFParser` (since it's a function or value it has to start
with a lower-case character; in this particular case the resulting
name is perhaps not-that elegant).

In [6]:
:t aRFParser

aRFParser :: ParserOptions

This parser is used with `readTableOpt` to read in the data. As the variable name
(`arfStream`) suggests, this supports a streaming approach
(i.e. is useful for handling large data sets, that you may not want to read
in all in one go),
built on the 
[Pipes](https://hackage.haskell.org/package/pipes-4.1.5/docs/Pipes-Tutorial.html)
library.

XXX 

In [7]:
arfStream = readTableOpt aRFParser "../data/arf.csv"

The type is intimidating...

In [8]:
:type arfStream

arfStream :: forall (m :: * -> *) (rs :: [*]). (ReadRec rs, MonadIO m) => Producer (Record rs) m ()

but fortunately we can ignore this and just 
convert it into an "in-memory" representation (in this case, an "Array of Structures")
using the `inCoreAoS` function.

In [9]:
-- signature needed for the type inference
arf <- inCoreAoS arfStream :: IO (Frame ARF)

In [10]:
:type arf

arf :: Frame ARF

XXX What can I do with this?

I can get back the column headers, in a round about way (for those people used to using 
Python or R):

In [11]:
columnHeaders (Proxy :: Proxy ARF)

["ENERG_LO","ENERG_HI","SPECRESP"]

This has actually worked out the column headers from the *type*, and not from
the value (since `arf` isn't passed to `columnHeaders`). 

XXX not sure where I'm going with this or if it's worth it

In [12]:
:info Record

type Record = Data.Vinyl.Core.Rec Data.Vinyl.Functor.Identity -- Defined in ‘Frames.Rec’

In [13]:
:type columnHeaders

columnHeaders :: forall (cs :: [*]) (proxy :: * -> *) (f :: * -> *). ColumnHeaders cs => proxy (Rec f cs) -> [String]

Almost certainly better to show data access and then perhaps some of the types
than the current approach!

In [14]:
:type view

view :: forall b a a' b'. FoldLike b a a' b b' -> a -> b

Argh: my eyes - `sPECRESP` is not a visually-engaging symbol...

In [15]:
:type view sPECRESP

view sPECRESP :: forall (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Record rs0 -> Double

In [16]:
:type view sPECRESP <$> arf

view sPECRESP <$> arf :: Frame Double

In [17]:
view sPECRESP <$> arf

No instance for (Show (Frame Double)) arising from a use of ‘print’
In a stmt of an interactive GHCi command: print it

So, this is "projecting" out the `SPECRESP` field from the row, creating a frame with only this content:

In [18]:
mapM_ print (take 3 (F.toList (view sPECRESP <$> arf)))

4.874343
14.82926
21.30229

In [19]:
-- since I'm going to use this a few times
printN n = mapM_ print . take n . F.toList

How about summary statistics? This can often achieved with a `fold`
over the structure:

In [20]:
:type L.fold
:type L.fold L.minimum
:type L.fold L.minimum (view sPECRESP <$> arf)

L.fold :: forall a b (f :: * -> *). Foldable f => Fold a b -> f a -> b

L.fold L.minimum :: forall a (f :: * -> *). (Foldable f, Ord a) => f a -> Maybe a

L.fold L.minimum (view sPECRESP <$> arf) :: Maybe Double

In [21]:
L.fold L.minimum (view sPECRESP <$> arf)
L.fold L.maximum (view sPECRESP <$> arf)

Just 0.5657126

Just 672.1996

Note that the following does *not* evaluate the structure twice (once for
`minimum` and once for `maximum`):

In [22]:
-- from the tutorial
minMax :: Ord a => L.Fold a (Maybe a, Maybe a)
minMax = (,) <$> L.minimum <*> L.maximum

In [23]:
L.fold minMax (view sPECRESP <$> arf)

(Just 0.5657126,Just 672.1996)

In [24]:
:type L.pretraverse sPECRESP

L.pretraverse sPECRESP :: forall r (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Fold Double r -> Fold (Record rs0) r

In [25]:
:type L.pretraverse sPECRESP minMax 

L.pretraverse sPECRESP minMax :: forall (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Fold (Record rs0) (Maybe Double, Maybe Double)

In [26]:
L.fold (L.pretraverse sPECRESP minMax) arf

(Just 0.5657126,Just 672.1996)

Note that in the following the columns include their names (or, rather, the fields
in each record). This can be compared to earlier when `view sPECRESP` was used to
"extract" out the contents of the `SPECRESP` field, but losing the name):

In [27]:
printN 3 arf

{ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}

In [28]:
printN 3 (view sPECRESP <$> arf)

4.874343
14.82926
21.30229

How about just selecting a single column but leaving the "named field" part:

In [29]:
:info SPECRESP

type SPECRESP = "SPECRESP" :-> Double -- Defined at :1:1

In [30]:
select (Proxy::Proxy '[SPECRESP]) $ frameRow arf 0

{SPECRESP :-> 4.874343}

In [31]:
sprespOnly :: ARF -> Record '[SPECRESP]
sprespOnly = rcast

In [32]:
printN 3 (fmap sprespOnly arf)

{SPECRESP :-> 4.874343}
{SPECRESP :-> 14.82926}
{SPECRESP :-> 21.30229}

In [33]:
nrows = frameLength arf
mapM_ (print . frameRow arf) [nrows - 3 .. nrows - 1]

{ENERG_LO :-> 10.97, ENERG_HI :-> 10.98, SPECRESP :-> 0.6049552}
{ENERG_LO :-> 10.98, ENERG_HI :-> 10.99, SPECRESP :-> 0.5852773}
{ENERG_LO :-> 10.99, ENERG_HI :-> 11.0, SPECRESP :-> 0.5657126}

In [34]:
-- subsets
mapM_ print (take 5 (filter ((>= 600) . view sPECRESP) (F.toList arf)))

{ENERG_LO :-> 1.18, ENERG_HI :-> 1.19, SPECRESP :-> 601.2171}
{ENERG_LO :-> 1.19, ENERG_HI :-> 1.2, SPECRESP :-> 604.2618}
{ENERG_LO :-> 1.2, ENERG_HI :-> 1.21, SPECRESP :-> 607.2027}
{ENERG_LO :-> 1.21, ENERG_HI :-> 1.22, SPECRESP :-> 610.0424}
{ENERG_LO :-> 1.22, ENERG_HI :-> 1.23, SPECRESP :-> 612.9407}

In [35]:
-- would be nice to have a HTML representation of the table;
-- how to get column names (see above)?
arf

No instance for (Show (Frame ARF)) arising from a use of ‘print’
In a stmt of an interactive GHCi command: print it

In [36]:
-- and how to extract the data to plot it? Frames has plot examples

In [37]:
:t recUncons

recUncons :: forall (s :: Symbol) a (rs :: [*]). Record ((s :-> a) : rs) -> (a, Record rs)

In [38]:
-- this gives the value, but not the name
(a1,b1) = recUncons (frameRow arf 0)
(a2,b2) = recUncons b1
(a3,b3) = recUncons b2

(a1,a2,a3)

(0.3,0.31,4.874343)

In [39]:
b1
b2
b3

{ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}

{SPECRESP :-> 4.874343}

{}

In [40]:
-- as expected, this falls over
recUncons b3

Couldn't match type ‘'[]’ with ‘(s0 :-> a) : rs’
Expected type: Record ((s0 :-> a) : rs)
  Actual type: Record '[]
Relevant bindings include it :: (a, Record rs) (bound at <interactive>:1:1)
In the first argument of ‘recUncons’, namely ‘b3’
In the expression: recUncons b3

In [41]:
showFields (frameRow arf 0)

["0.3","0.31","4.874343"]

In [42]:
toVinyl (frameRow arf 0)

{0.3, 0.31, 4.874343}

In [43]:
:type toVinyl (frameRow arf 0)

toVinyl (frameRow arf 0) :: Rec Identity (UnColumn '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double])

In [44]:
:type frameRow

frameRow :: forall r. Frame r -> Int -> r

In [45]:
:info ARF

type ARF = Record '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double] -- Defined at :1:1

Let's leave making a nice HTML table for now.

From `Frames.Col`:

```
instance forall s a. (KnownSymbol s, Show a) => Show (s :-> a) where
  show (Col x) = symbolVal (Proxy::Proxy s)++" :-> "++show x
```

In [46]:
:info ENERGLO

type ENERGLO = "ENERG_LO" :-> Double -- Defined at :1:1

In [47]:
type ENERGY = "ENERGY" :-> Double

In [48]:
rnew :: Record '[ENERGY, ENERGLO, ENERGHI, SPECRESP]
rnew = frameCons (pure 0.305) (frameRow arf 0)

In [49]:
rnew

{ENERGY :-> 0.305, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}

In [50]:
:info ARF

type ARF = Record '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double] -- Defined at :1:1

In [51]:
:type rnew

rnew :: Record '[ENERGY, ENERGLO, ENERGHI, SPECRESP]

In [52]:
-- would it be easy to add in the column after ENERGHI, say?
type ARFmod = Record '[ENERGY, ENERGLO, ENERGHI, SPECRESP]

In [53]:
-- Ideally would only require the lo/hi energy fields;
-- => addEnergyCol2
addEnergyCol :: Record '[ENERGLO, ENERGHI, SPECRESP] -> ARFmod
addEnergyCol r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
      emid = (elo + ehi) / 2
  in frameCons (pure emid) r

In [54]:
addEnergyCol2 :: 
  (ENERGLO ∈ rs, ENERGHI ∈ rs) 
  => Record rs
  -> Record (ENERGY ': rs)
addEnergyCol2 r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
      emid = (elo + ehi) / 2
  in frameCons (pure emid) r

In [55]:
:type frameCons

frameCons :: forall (f :: * -> *) a (rs :: [*]) (s :: Symbol). Functor f => f a -> Rec f rs -> Rec f ((s :-> a) : rs)

In [56]:
{-

Not sure this is a worthwhile avenue

-- try and avoid types where necessary; I was hoping just to be able to give
-- a type to the new column, but it isn't obvious if it's possible...
addEnergyCol3 r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
      emid = (elo + ehi) / 2
      newcol :: ENERGY ????XXXXX what goes here
      newcol = pure emid
  in frameCons newcol r
  
  
-}  

In [57]:
addEnergy :: Frame ARF -> Frame ARFmod
addEnergy = fmap addEnergyCol

In [58]:
import qualified Pipes.Prelude as P
import Pipes hiding (Proxy)

In [59]:
{-

writers :: (Occupation ∈ rs, Monad m) => Pipe (Record rs) (Record rs) m r
writers = P.filter ((== "writer") . view occupation)

-}

-- for now be very specific with the types
addEnergyP :: Monad m => Pipe ARF ARFmod m r
addEnergyP = P.map addEnergyCol

In [60]:
runEffect (arfStream >-> addEnergyP >-> P.take 6 >-> P.print)

{ENERGY :-> 0.305, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{ENERGY :-> 0.315, ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{ENERGY :-> 0.325, ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}
{ENERGY :-> 0.335, ENERG_LO :-> 0.33, ENERG_HI :-> 0.34, SPECRESP :-> 28.51495}
{ENERGY :-> 0.345, ENERG_LO :-> 0.34, ENERG_HI :-> 0.35, SPECRESP :-> 35.39883}
{ENERGY :-> 0.355, ENERG_LO :-> 0.35, ENERG_HI :-> 0.36, SPECRESP :-> 41.54232}

In [61]:
arfMod = addEnergy arf

In [62]:
printN 6 arfMod

{ENERGY :-> 0.305, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{ENERGY :-> 0.315, ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{ENERGY :-> 0.325, ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}
{ENERGY :-> 0.335, ENERG_LO :-> 0.33, ENERG_HI :-> 0.34, SPECRESP :-> 28.51495}
{ENERGY :-> 0.345, ENERG_LO :-> 0.34, ENERG_HI :-> 0.35, SPECRESP :-> 35.39883}
{ENERGY :-> 0.355, ENERG_LO :-> 0.35, ENERG_HI :-> 0.36, SPECRESP :-> 41.54232}

Can I write a model function that requires an ARF-like record and calculates a model?

In [63]:
-- Note, since the "columns" are typed, in that SPECRESP/...
-- are Doubles, this is not polymorphic.
--
-- This is an "integrated" model
powerLaw1 ::   
  (ENERGLO ∈ rs, ENERGHI ∈ rs, SPECRESP ∈ rs) 
  => Double      -- ^ amplitude at 1 keV
  -> Double      -- ^ gamma
  -> Record rs
  -> Double      -- ^ units of the amplitude * Energy
powerLaw1 norm gamma rs = 
  let elo = view eNERGLO rs
      ehi = view eNERGHI rs
      p   = 1 - gamma
      val = if gamma == 1
            then log (ehi/elo)
            else (ehi**p - elo**p) / p
             
  in norm * val

Perhaps I want to add the model value to the record rather than just return a `Double`;
in this case I think the interface would be that there's a function to do this,
and it accepts a generic model (with an interface more like the ones shown
before, i.e. something that doesn't know about records).

In [64]:
type IModelFunc = Double -> Double -> Double

type IModel = "IModel" :-> Double

In [65]:
evalModel :: 
  (ENERGLO ∈ rs, ENERGHI ∈ rs, SPECRESP ∈ rs) 
  => IModelFunc
  -> Record rs
  -> Double
evalModel f r = 
  let elo = view eNERGLO r
      ehi = view eNERGHI r
  in f elo ehi
  
addModel :: 
  (ENERGLO ∈ rs, ENERGHI ∈ rs, SPECRESP ∈ rs) 
  => IModelFunc
  -> Record rs
  -> Record (IModel ': rs)
addModel f r = frameCons (pure (evalModel f r)) r

In [66]:
powerLaw :: 
  Double         -- ^ normalization 
  -> Double      -- ^ gamma 
  -> IModelFunc
powerLaw norm gamma elo ehi | gamma == 1 = norm * log (ehi / elo)
                            | otherwise  = let p = 1 - gamma
                                           in norm * (ehi**p - elo**p) / p

In [67]:
model = fmap (addModel (powerLaw 1.0 2.0)) arf

In [68]:
printN 6 model

{IModel :-> 0.10752688172043001, ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{IModel :-> 0.10080645161290347, ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{IModel :-> 9.469696969696972e-2, ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}
{IModel :-> 8.912655971479522e-2, ENERG_LO :-> 0.33, ENERG_HI :-> 0.34, SPECRESP :-> 28.51495}
{IModel :-> 8.403361344537785e-2, ENERG_LO :-> 0.34, ENERG_HI :-> 0.35, SPECRESP :-> 35.39883}
{IModel :-> 7.936507936507953e-2, ENERG_LO :-> 0.35, ENERG_HI :-> 0.36, SPECRESP :-> 41.54232}

Let's just get the energy and integrated-model value:

In [69]:
-- as no longer have a singleton list, do not need the quote before the []
-- but leave in for now.
--
smodel = fmap (select (Proxy::Proxy '[ENERGY, IModel]) . addEnergyCol2) model

In [70]:
printN 3 smodel

{ENERGY :-> 0.305, IModel :-> 0.10752688172043001}
{ENERGY :-> 0.315, IModel :-> 0.10080645161290347}
{ENERGY :-> 0.325, IModel :-> 9.469696969696972e-2}

So, how do I access these new columns? The `tableTypes` routine set up accessor
symbols - e.g. `eNERG_LO` - but I don't have them here.

In [71]:
view eNERGLO (frameRow arf 0)

0.3

In [72]:
eNERGY = rlens (Proxy :: Proxy ENERGY)
iModel = rlens (Proxy :: Proxy IModel)

In [73]:
view eNERGY (frameRow smodel 0)
view iModel (frameRow smodel 0)

0.305

0.10752688172043001

XXX can I get a vector out? Do I want to here?

XXX plot the data

In [74]:
import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Diagrams

-- Hide view since I want to use the Frames-provided version here
import Graphics.Rendering.Chart.Easy hiding (view)

instance IHaskellDisplay (Renderable a) where
    display renderable = renderableToSVG renderable 450 300 >>= display . fst

In [75]:
displayIModel :: (ENERGY ∈ rs, IModel ∈ rs) => Frame (Record rs) -> Renderable ()
displayIModel frm = toRenderable (do
  let getCS rs = (view eNERGY rs, LogValue (view iModel rs))
      cs = F.toList (fmap getCS frm) 
      
  layout_title .= "Model"
  layout_x_axis . laxis_title .= "E (keV)"
  layout_y_axis . laxis_title .= "photon/cm^2/s"
  
  plot (line "" [cs])
  )

In [76]:
displayIModel smodel

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
    "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" width="450.0" height="300.0" font-size="1" viewBox="0 0 450 300" stroke="rgb(0,0,0)" stroke-opacity="1">
    <g>
        <g fill="rgb(0,0,0)" fill-opacity="0.0">
            <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="1.4696938456699067" stroke-linecap="butt" stroke-linejoin="miter" font-size="1.0em" stroke-miterlimit="10.0">
                <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                    <g stroke="rgb(0,0,0)" stroke-opacity="0.0" fill="rgb(255,255,255)" fill-opacity="1.0" stroke-width="0.0" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                        <g fill="rgb(255,255,255)" fill-opacity="1.0">
                            <g fill="rgb(255,255,255)" fill-opacity="1.0">
                                <path d="M 0.0,2.842170943040401e-14 v 299.99999999999994 h 449.9999999999999 v -299.99999999999994 Z" />
                            </g>
                        </g>
                    </g>
                    <g stroke="rgb(0,0,0)" stroke-opacity="0.0" fill="rgb(0,0,0)" fill-opacity="1.0" stroke-width="0.0" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                        <g fill="rgb(0,0,0)" fill-opacity="1.0">
                            <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                </g>
                                <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                    <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                    </g>
                                    <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                        <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                            <path d="M 204.94096509240242,14.101642710472305 ZM 206.126796714579,4.060574948665326 h 2.4024640657084184 l 1.678644763860369,4.512320328542093 l 0.6006160164271046,1.7556468172484594 h 6.160164271047226e-2 l 0.6006160164271046,-1.7556468172484594 l 1.6324435318275148,-4.512320328542093 h 2.3870636550308 v 10.04106776180698 h -2.109856262833675 v -3.680698151950718 c 0.0,-0.3696098562628336 3.080082135523613e-2,-0.8162217659137574 3.080082135523613e-2 -0.8162217659137574c 3.080082135523613e-2,-0.4466119096509239 7.700205338809034e-2,-0.9009240246406568 7.700205338809034e-2 -0.9009240246406568c 4.62012320328542e-2,-0.45431211498973295 9.24024640657084e-2,-0.8932238193018478 9.24024640657084e-2 -0.8932238193018478c 4.62012320328542e-2,-0.43891170431211485 9.24024640657084e-2,-0.8085215605749484 9.24024640657084e-2 -0.8085215605749484h -6.160164271047226e-2 l -0.8316221765913755,2.325462012320328 l -1.5400410677618064,3.8963039014373706 h -0.9086242299794659 l -1.5400410677618064,-3.8963039014373706 l -0.8008213552361394,-2.325462012320328 h -6.160164271047226e-2 c 4.62012320328542e-2,0.3696098562628336 9.24024640657084e-2,0.8085215605749484 9.24024640657084e-2 0.8085215605749484c 4.62012320328542e-2,0.43891170431211485 8.470225872689936e-2,0.8932238193018478 8.470225872689936e-2 0.8932238193018478c 3.850102669404517e-2,0.45431211498973295 6.93018480492813e-2,0.9009240246406568 6.93018480492813e-2 0.9009240246406568c 3.080082135523613e-2,0.4466119096509239 3.080082135523613e-2,0.8162217659137574 3.080082135523613e-2 0.8162217659137574v 3.680698151950718 h -2.079055441478439 v -10.04106776180698 ZM 216.67607802874738,14.101642710472305 ZM 217.23049281314164,10.282340862423025 c 0.0,-0.9548254620123201 0.30800821355236135,-1.701745379876796 0.30800821355236135 -1.701745

XXX TODO: do a "per kev" version. for that really need the energ_lo/hi bins ...

XXX could there be a simple "frame plot" command that takes two columns and plots them, using
XXX column headers as labels

In [77]:
displayModel :: (ENERGLO ∈ rs, ENERGHI ∈ rs, IModel ∈ rs) => Frame (Record rs) -> Renderable ()
displayModel frm = toRenderable (do
  let getCS rs = let elo = view eNERGLO rs
                     ehi = view eNERGHI rs
                     mdl = view iModel rs 
                 in (0.5*(elo+ehi), LogValue (mdl / (ehi-elo)))
                 
      cs = F.toList (fmap getCS frm) 
      
  layout_title .= "Model"
  layout_x_axis . laxis_title .= "E (keV)"
  layout_y_axis . laxis_title .= "photon/cm^2/s/keV"
  
  plot (line "" [cs])
  )

In [78]:
displayModel model

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
    "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" width="450.0" height="300.0" font-size="1" viewBox="0 0 450 300" stroke="rgb(0,0,0)" stroke-opacity="1">
    <g>
        <g fill="rgb(0,0,0)" fill-opacity="0.0">
            <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="1.4696938456699067" stroke-linecap="butt" stroke-linejoin="miter" font-size="1.0em" stroke-miterlimit="10.0">
                <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                    <g stroke="rgb(0,0,0)" stroke-opacity="0.0" fill="rgb(255,255,255)" fill-opacity="1.0" stroke-width="0.0" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                        <g fill="rgb(255,255,255)" fill-opacity="1.0">
                            <g fill="rgb(255,255,255)" fill-opacity="1.0">
                                <path d="M 0.0,2.842170943040401e-14 v 299.99999999999994 h 449.9999999999999 v -299.99999999999994 Z" />
                            </g>
                        </g>
                    </g>
                    <g stroke="rgb(0,0,0)" stroke-opacity="0.0" fill="rgb(0,0,0)" fill-opacity="1.0" stroke-width="0.0" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                        <g fill="rgb(0,0,0)" fill-opacity="1.0">
                            <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                </g>
                                <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                    <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                    </g>
                                    <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                        <g fill="rgb(0,0,0)" fill-opacity="1.0">
                                            <path d="M 204.94096509240242,14.101642710472305 ZM 206.126796714579,4.060574948665326 h 2.4024640657084184 l 1.678644763860369,4.512320328542093 l 0.6006160164271046,1.7556468172484594 h 6.160164271047226e-2 l 0.6006160164271046,-1.7556468172484594 l 1.6324435318275148,-4.512320328542093 h 2.3870636550308 v 10.04106776180698 h -2.109856262833675 v -3.680698151950718 c 0.0,-0.3696098562628336 3.080082135523613e-2,-0.8162217659137574 3.080082135523613e-2 -0.8162217659137574c 3.080082135523613e-2,-0.4466119096509239 7.700205338809034e-2,-0.9009240246406568 7.700205338809034e-2 -0.9009240246406568c 4.62012320328542e-2,-0.45431211498973295 9.24024640657084e-2,-0.8932238193018478 9.24024640657084e-2 -0.8932238193018478c 4.62012320328542e-2,-0.43891170431211485 9.24024640657084e-2,-0.8085215605749484 9.24024640657084e-2 -0.8085215605749484h -6.160164271047226e-2 l -0.8316221765913755,2.325462012320328 l -1.5400410677618064,3.8963039014373706 h -0.9086242299794659 l -1.5400410677618064,-3.8963039014373706 l -0.8008213552361394,-2.325462012320328 h -6.160164271047226e-2 c 4.62012320328542e-2,0.3696098562628336 9.24024640657084e-2,0.8085215605749484 9.24024640657084e-2 0.8085215605749484c 4.62012320328542e-2,0.43891170431211485 8.470225872689936e-2,0.8932238193018478 8.470225872689936e-2 0.8932238193018478c 3.850102669404517e-2,0.45431211498973295 6.93018480492813e-2,0.9009240246406568 6.93018480492813e-2 0.9009240246406568c 3.080082135523613e-2,0.4466119096509239 3.080082135523613e-2,0.8162217659137574 3.080082135523613e-2 0.8162217659137574v 3.680698151950718 h -2.079055441478439 v -10.04106776180698 ZM 216.67607802874738,14.101642710472305 ZM 217.23049281314164,10.282340862423025 c 0.0,-0.9548254620123201 0.30800821355236135,-1.701745379876796 0.30800821355236135 -1.701745

The type safety comes into play when you try to apply the wrong routine for a frame
(or give the wrong frame to a plot function):

In [79]:
displayModel smodel

No instance for (RElem ("ENERG_LO" :-> Double) '[] (Data.Vinyl.TypeLevel.RIndex ("ENERG_LO" :-> Double) '[])) arising from a use of ‘displayModel’
In the expression: displayModel smodel
In an equation for ‘it’: it = displayModel smodel

In [80]:
displayIModel model

No instance for (RElem ("ENERGY" :-> Double) '[] (Data.Vinyl.TypeLevel.RIndex ("ENERGY" :-> Double) '[])) arising from a use of ‘displayIModel’
In the expression: displayIModel model
In an equation for ‘it’: it = displayIModel model

In [81]:
import GHC.TypeLits (KnownSymbol)

In [82]:
-- | Plot up the first two columns (as x and y)
--
--   column selection is left to the caller - e.g. via judicious calls to `select`
--
splot :: 
  (PlotValue a, PlotValue b, KnownSymbol sx, KnownSymbol sy, ColumnHeaders rs) 
  => (x -> a)
  -> (y -> b)
  -> Frame (Record (sx :-> x ': sy :-> y ': rs))
  -> Renderable ()
splot px py frm = toRenderable (do
  let getxy rs = let (x, yrs) = recUncons rs
                     (y, _)   = recUncons yrs
                 in (px x, py y)
                 
      cs = F.toList (fmap getxy frm)
      
      (xlbl:ylbl:_) = columnHeaders frm

  layout_x_axis . laxis_title .= xlbl
  layout_y_axis . laxis_title .= ylbl
  plot (line "" [cs])
  )

In [83]:
splot id id (fmap (select [pr|ENERGY,SPECRESP|]) arfMod)

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
    "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" width="450.0" height="300.0" font-size="1" viewBox="0 0 450 300" stroke="rgb(0,0,0)" stroke-opacity="1">
    <g>
        <g fill="rgb(0,0,0)" fill-opacity="0.0">
            <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="1.4696938456699067" stroke-linecap="butt" stroke-linejoin="miter" font-size="1.0em" stroke-miterlimit="10.0">
                <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                    <g stroke="rgb(0,0,0)" stroke-opacity="0.0" fill="rgb(255,255,255)" fill-opacity="1.0" stroke-width="0.0" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                        <g fill="rgb(255,255,255)" fill-opacity="1.0">
                            <g fill="rgb(255,255,255)" fill-opacity="1.0">
                                <path d="M 0.0,2.842170943040401e-14 v 299.99999999999994 h 449.9999999999999 v -299.99999999999994 Z" />
                            </g>
                        </g>
                    </g>
                    <g>
                        <g>
                            <g stroke="rgb(211,211,211)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="miter" stroke-dasharray="4.999999999999999,4.999999999999999" stroke-dashoffset="0.0">
                                <g>
                                    <path d="M 55.021008403361336,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 118.42261904761901,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 181.8242296918767,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 245.2258403361344,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 308.62745098039204,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 372.0290616246498,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 435.4306722689075,15.000000000000025 v 244.99999999999994 " />
                                </g>
                            </g>
                            <g stroke="rgb(211,211,211)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="miter" stroke-dasharray="4.999999999999999,4.999999999999999" stroke-dashoffset="0.0">
                                <g>
                                    <path d="M 55.021008403361336,259.99999999999994 h 380.4096638655461 " />
                                </g>
                                <g>
                                    <path d="M 55.021008403361336,198.75 h 380.4096638655461 " />
                                </g>
                                <g>
                                    <path d="M 55.021008403361336,137.5 h 380.4096638655461 " />
                                </g>
                                <g>
                                    <path d="M 55.021008403361336,76.25000000000001 h 380.4096638655461 " />
                                </g>
                                <g>
                                    <path d="M 55.02100840

In [84]:
splot LogValue LogValue (fmap (select [pr|ENERGY,IModel|]) smodel)

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
    "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" width="450.0" height="300.0" font-size="1" viewBox="0 0 450 300" stroke="rgb(0,0,0)" stroke-opacity="1">
    <g>
        <g fill="rgb(0,0,0)" fill-opacity="0.0">
            <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="1.4696938456699067" stroke-linecap="butt" stroke-linejoin="miter" font-size="1.0em" stroke-miterlimit="10.0">
                <g stroke="rgb(0,0,0)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                    <g stroke="rgb(0,0,0)" stroke-opacity="0.0" fill="rgb(255,255,255)" fill-opacity="1.0" stroke-width="0.0" stroke-linecap="butt" stroke-linejoin="bevel" stroke-dasharray="" stroke-dashoffset="0.0">
                        <g fill="rgb(255,255,255)" fill-opacity="1.0">
                            <g fill="rgb(255,255,255)" fill-opacity="1.0">
                                <path d="M 0.0,2.842170943040401e-14 v 299.99999999999994 h 449.9999999999999 v -299.99999999999994 Z" />
                            </g>
                        </g>
                    </g>
                    <g>
                        <g>
                            <g stroke="rgb(211,211,211)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="miter" stroke-dasharray="4.999999999999999,4.999999999999999" stroke-dashoffset="0.0">
                                <g>
                                    <path d="M 72.86764705882352,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 166.68816768123577,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 207.0944665795501,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 207.0944665795501,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 300.9149872019624,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 341.3212861002768,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 341.3212861002768,15.000000000000025 v 244.99999999999994 " />
                                </g>
                                <g>
                                    <path d="M 435.141806722689,15.000000000000025 v 244.99999999999994 " />
                                </g>
                            </g>
                            <g stroke="rgb(211,211,211)" stroke-opacity="1.0" stroke-width="0.9999999999999998" stroke-linecap="butt" stroke-linejoin="miter" stroke-dasharray="4.999999999999999,4.999999999999999" stroke-dashoffset="0.0">
                                <g>
                                    <path d="M 72.86764705882352,259.99999999999994 h 362.2741596638655 " />
                                </g>
                                <g>
                                    <path d="M 72.86764705882352,210.99999999999994 h 362.2741596638655 " />
                                </g>
                                <g>
                                    <path d="M 72.86764705882352,210.99999999999994 h 362.2741596638655 " />
                                </g>
                                <g>
                                    <p